In [1]:
import openai
openai.api_key = "your-openai-key"

In [2]:
from openai import OpenAI
client = OpenAI(api_key="your-openai-key")


try:
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": "Hello! Can you read this?"}]
    )
    print("API Key is working! ✔️")
    print("Response:", response.choices[0].message.content)

except Exception as e:
    print("API Key error ❌")
    print(e)



API Key is working! ✔️
Response: Hello! Yes, I can read your message. How can I assist you today?


In [3]:
!pip install chromadb pymupdf langchain openai langchain-openai


In [4]:

import fitz

def extract_pdf_text(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    doc.close()
    return text

pdf_path = "/content/ecb.wp2409~79137cac1e.en.pdf"
raw_text = extract_pdf_text(pdf_path)
len(raw_text)


95594

In [8]:
!pip install langchain-text-splitters

In [9]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=150
)

chunks = splitter.split_text(raw_text)
len(chunks)


145

In [12]:
!pip install -q langchain langchain-openai langchain-community chromadb


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 53.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
google-adk 1.20.0 requires opentelemetry-api<=1.37.0,>=1.37.0, but you have opentelemetry-api 1.39.1 which is incompatible.
google-adk 1.20.0 requires opentelemetry-sdk<=1.37.0,>=1.37.0, but you have opentelemetry-sdk 1.39.1 which is incompatible.
opentelemetry-exporter-otlp-proto-http 1.37.0 requires opentelemetry-exporter-otlp-proto-common==1.37.0, but you have opentelemetry-exporter-otlp-proto-common 1.39.1 whi

In [13]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma


embeddings = OpenAIEmbeddings(model="text-embedding-3-small", api_key="your-openai-key")

vectordb = Chroma.from_texts(
    texts=chunks,
    embedding=embeddings,
    collection_name="pdf_rag"
)


In [14]:
retriever = vectordb.as_retriever(search_type="similarity", search_kwargs={"k": 4})


In [19]:
from openai import OpenAI
client = OpenAI(api_key="your-openai-key")

def ask_question(query):
    # retrieve docs with the new API
    docs = retriever.invoke(query)

    # extract text from docs
    context = "\n\n".join([d.page_content for d in docs])

    prompt = f"""
    You are a helpful assistant. Answer *only* using the information provided in the context.
    If the answer is not in the context, say: "The answer is not in the document."

    CONTEXT:
    {context}

    QUESTION:
    {query}
    """

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}]
    )

    return response.choices[0].message.content


In [20]:
ask_question("main focus of the document in 2 words")


'Credit growth'

In [21]:
!pip install ragas datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 419.9/419.9 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.9/160.9 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 47.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 358.8/358.8 kB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.4/226.4 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 44.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 52.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 30.3 MB/s eta 0:00:00
  Attempting uninstall: jiter
    Found existing installation: jiter 0.12.0
    Uninstalling jiter-0.12.0:
      Successfully uninstalled jiter-0.12.0


In [42]:
import pandas as pd

# ✏️ Put ground-truth answers here (manually or auto-generated)
ground_truths = {
    "Summarize the document?": "The document discusses the Spanish Central Credit Register (CIR), which is managed by the Banco de España and contains detailed monthly information on loans exceeding 6,000 euros to non-financial firms in Spain since 1995. It notes that due to the low reporting threshold, nearly all firms with outstanding bank debt are included in the CIR. Additionally, the document lists various statistical analyses related to loan granting and macroeconomic variables, presenting findings such as R-squared and adjusted R-squared values from different regressions categorized by loan type. It also acknowledges contributions from several individuals and institutions involved in the research.",
    "Who is the author?": "The answer is not in the document"
}

eval_questions = [
    "Summarize the document?",
    "Who is the author?"
]

eval_data = []

for q in eval_questions:
    answer = ask_question(q)  # your RAG output
    contexts = [str(doc.page_content) for doc in retriever.invoke(q)]
    ground_truth = ground_truths[q]          # required for answer_correctness
    reference = ground_truths[q]             # required for context_precision & recall

    eval_data.append({
        "question": str(q),
        "answer": str(answer),
        "contexts": contexts,
        "ground_truth": str(ground_truth),
        "reference": str(reference)
    })

df = pd.DataFrame(eval_data)
df


,question,answer,contexts,ground_truth,reference
0,Summarize the document?,The document discusses the Spanish Central Cre...,"[sample. \nGiven \nthis \nconstraint, \nin \nw...",The document discusses the Spanish Central Cre...,The document discusses the Spanish Central Cre...
1,Who is the author?,The answer is not in the document.,"[All rights reserved. Any reproduction, public...",The answer is not in the document,The answer is not in the document


In [41]:
ask_question("name the author of the document")

'The answer is not in the document.'

In [43]:
from datasets import Dataset

ragas_dataset = Dataset.from_pandas(df)


In [45]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    answer_correctness,
    context_precision,
    context_recall
)
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o-mini"
)

results = evaluate(
    ragas_dataset,
    metrics=[
        faithfulness,
    answer_relevancy,
    answer_correctness,
    context_precision,
    context_recall
    ],
    llm=llm
)

results


Evaluating:   0%|          | 0/10 [00:00<?, ?it/s]

ERROR:ragas.executor:Exception raised in Job[0]: AttributeError('OpenAI' object has no attribute 'generate')
ERROR:ragas.executor:Exception raised in Job[1]: AttributeError('OpenAI' object has no attribute 'generate')
ERROR:ragas.executor:Exception raised in Job[5]: AttributeError('OpenAI' object has no attribute 'generate')
ERROR:ragas.executor:Exception raised in Job[6]: AttributeError('OpenAI' object has no attribute 'generate')


{'faithfulness': nan, 'answer_relevancy': nan, 'answer_correctness': 0.8553, 'context_precision': 1.0000, 'context_recall': 0.5000}

In [53]:
while True:
    query = input("Enter your query: ")
    if query.lower() == "exit":
        print("Exiting...")
        break

    print(ask_question(query))


Enter your query: what does this pdf conveys in 30 words
This PDF discusses the ECB Working Paper Series, providing statistics on credit growth and macroeconomic variables, along with detailed information on loan datasets and analysis of bank characteristics.
Enter your query: exit
Exiting...
